# Lab 9: Association Rules


This lab material is largely self-contained. We assume that every student has already taken STAT7008 or knows some basic operations of Python. Noet that you may use Anaconda to run the .ipynb file. For the installation of Anaconda, please see https://conda.io/docs/user-guide/install/index.html.

### Purpose

In Lab 9, you will learn how to:

a. do association rules.

### Useful libraries for this Lab

a. numpy, for data array. 

b. mlxtend, for association rules.

d. sklearn, for data and preprocessing.

e. os, for the working directory. 

In [1]:
import numpy as np
import os

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

wd = os.getcwd() # Set your working directory. 
print wd

/home/renjielu/PycharmProjects/DM8017/DM_Lab9


In Lab 9, we use the online retail data set, provided by the UC irvine machine learning repository which contains 426 data sets. Its website is https://archive.ics.uci.edu/ml/index.php. The first five records are listed below. 

In order to simplify the whole analysis in this lab, we remove the rows who have no invoice number, and the credit transactions whose invoice numbers contain C. 

Also, we remove the spaces in the elements of the descriptions in the data. 

In [13]:
df = pd.read_excel(wd+'/Online Retail.xlsx')
print df.head()
print len(df)
print '\n'

print 'Column names: ', list(df)

df['Description'] = df['Description'].str.strip() # remove the spaces.
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # remove the missing values.
df['InvoiceNo'] = df['InvoiceNo'].astype('str') 
df = df[~df['InvoiceNo'].str.contains('C')]

print len(df)

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
541909


Column names:  [u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate', u'UnitPrice', u'CustomerID', u'Country']
532621


For simplicity, we only focus on the sales of France in this lab. Before conducting association rules, we need to transform the data by using multi-hot encoder. Specifically, for each invoice,  we assign 1 or 0 (occurence/non-occurrence) to the correspodning items, without considering their purchase quantity. For example, suppose that we have total three goods in a store: item A, item B and item C, and there are two items (A and B) in the invoice. Then, the transformed data is [1,1,0].   

Note that we remove the postage variable since it is activated for every good (recall that this is an online retail data set).

In [24]:
Fbasket = (df[df['Country'] =="France"].
          groupby(['InvoiceNo', 'Description'])['Quantity'].
          sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

Fbasket_sets = Fbasket.applymap(encode_units) # keep all values must be 1 or 0.
Fbasket_sets.drop('POSTAGE', inplace=True, axis=1)

print Fbasket.head()


Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                          0.0                         0.0   
536852                          0.0                         0.0   
536974                          0.0                         0.0   
537065                          0.0                         0.0   
537463                          0.0                         0.0   

Description  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                                                 
536370                             0.0                              0.0   
536852                             0.0                              0.0   
536974                             0.0                              0.0   
537065                             0.0                              0.0   
537463                             0.0                              0.0   

Desc

We set the minimum support in the Apriori algorithm as 0.07. Then, the rules are given as follows. 

In this lab, we show the rules whose lift and confidence are larger than 8 and 0.8, respectively.

In the final rules, it seems that the green and red alarm clocks are purchased together.

In [31]:
frequent_itemsets = apriori(Fbasket_sets, min_support=0.07, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print rules.head()

final_rules = rules[ (rules['lift'] >= 8) & (rules['confidence'] >= 0.8) ]

print '\n'
print 'OUR FINAL RULES'

print final_rules

                                         antecedants  \
0                 (PLASTERS IN TIN WOODLAND ANIMALS)   
1                         (PLASTERS IN TIN SPACEBOY)   
2                      (SET/6 RED SPOTTY PAPER CUPS)   
3               (SET/20 RED RETROSPOT PAPER NAPKINS)   
4  (SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...   

                            consequents   support  confidence      lift  
0            (PLASTERS IN TIN SPACEBOY)  0.170918    0.611940  4.442233  
1    (PLASTERS IN TIN WOODLAND ANIMALS)  0.137755    0.759259  4.442233  
2  (SET/20 RED RETROSPOT PAPER NAPKINS)  0.137755    0.740741  5.584046  
3         (SET/6 RED SPOTTY PAPER CUPS)  0.132653    0.769231  5.584046  
4  (SET/20 RED RETROSPOT PAPER NAPKINS)  0.122449    0.812500  6.125000  


OUR FINAL RULES
                     antecedants                   consequents   support  \
20    (ALARM CLOCK BAKELIKE RED)  (ALARM CLOCK BAKELIKE GREEN)  0.094388   
21  (ALARM CLOCK BAKELIKE GREEN)    (ALARM CLOCK 